In [63]:
import torch

# Dummy data
B, T, C = 2, 4, 3
x = torch.randn(B, T, C)

# Padding mask: True = padded, False = valid
padding_mask = torch.tensor([
    [[False], [False], [True], [True]],
    [[False], [False], [False], [True]]
])  # shape: (2, 4, 1)


In [64]:
padding_mask, x

(tensor([[[False],
          [False],
          [ True],
          [ True]],
 
         [[False],
          [False],
          [False],
          [ True]]]),
 tensor([[[ 0.5609, -0.0648, -1.4102],
          [-0.4475, -0.9958,  3.3383],
          [ 1.6151,  0.5140, -1.1407],
          [ 0.1531,  0.0459, -0.5662]],
 
         [[ 1.1257,  1.3748,  1.2894],
          [ 1.0034,  0.4477, -0.8479],
          [ 1.5721,  0.1862, -0.0162],
          [-1.0669,  0.2631,  0.1311]]]))

In [65]:
B, T, C = x.size()
padding_mask = padding_mask.squeeze(-1)
time_indices = torch.arange(T).view(1, T, 1).expand(B, T, C)
expanded_mask = padding_mask.unsqueeze(-1).expand(B, T, C)
masked_indices = time_indices.masked_fill(expanded_mask, -1)
last_valid_t = masked_indices.max(dim=1).values
gather_index = last_valid_t.unsqueeze(1)  # (B, 1, C)
x = torch.gather(x, dim=1, index=gather_index)  # (B, 1, C)
x = x.squeeze(1)  # (B, C)
x

tensor([[-0.4475, -0.9958,  3.3383],
        [ 1.5721,  0.1862, -0.0162]])

In [66]:
gather_index

tensor([[[1, 1, 1]],

        [[2, 2, 2]]])